## 4.2.1 은닉층 이후 계산의 문제점
은닉층 이후에서 계산이 오래 걸리는 곳    
- 은닉층의 뉴런과 가중치 행렬(W_out)의 곱
- Softmax 계층의 계산

## 4.2.2 다중 분류에서 이진 분류로
- 네거티브 샘플링 기법: 다중 분류를 이진 분류로 근사하는 것.

## 4.2.3 시그모이드 함수와 교차 엔트로피 오차
## 4.2.4 다중분류에서 이진분류로 (구현)

In [1]:
import sys
sys.path.append('..')
from common.np import *  # import numpy as np
from common.layers import Embedding, SigmoidWithLoss
import collections


class EmbeddingDot:
    def __init__(self, W):
        self.embed = Embedding(W)
        self.params = self.embed.params
        self.grads = self.embed.grads
        self.cache = None

    def forward(self, h, idx):
        target_W = self.embed.forward(idx)
        out = np.sum(target_W * h, axis=1)

        self.cache = (h, target_W)
        return out

    def backward(self, dout):
        h, target_W = self.cache
        dout = dout.reshape(dout.shape[0], 1)

        dtarget_W = dout * h
        self.embed.backward(dtarget_W)
        dh = dout * target_W
        return dh

- EmbeddingDot 클래스에는 총 4개의 인스턴스 변수(embed, params, grads, cache)
   - embed는 Embedding계층을, cache는 순전파 시 계산 결과를 잠시 유지하기 위한 변수.    
- forward(h, idx) 메서드는 인수로 은닉층 뉴런(h)와 단어ID의 넘파이 배열(idx)를 받음. 배열로 받는 이유는 데이터를 한꺼번에 처리하는 '미니배치 처리'를 가정했기 때문.

## 4.2.5 네거티브 샘플링
- 정답(긍정적 예)에 대해서는 sigmoid계층의 출력을 1에 가깝게 만들고
- 오답(부정적 예)에 대해서는 sigmoid계층의 출력을 0에 가깝게 만들어야 함.
- 적은 수의 부정적 예를 샘플링해 사용. >> 네거티브 샘플링
- 네거티브 샘플링 기법은 긍정적 예를 타깃으로 한 경우의 손실을 구하고,
- 동시에 부정적 예를 몇 개 샘플링(선별)하여, 그 부정적 예에 대해서도 마찬가지로 손실을 구함.
- 그리고 각각의 데이터(긍정적 예와 샘플링된 부정적 예)의 손실을 더한 값을 최종 손실로 함.

## 4.2.6 네거티브 샘플링의 샘플링 기법
- 말뭉치의 통계 데이터를 기초로 샘플링. 
- 말뭉치에서 자주 등장하는 단어를 많이 추출하고 드물게 등장하는 단어를 적게 추출하는 것

In [1]:
# 확률분포에 따라 샘플링.
# np.random.choice() 무작위 샘플링 용도

import numpy as np

# 0에서 9까지의 숫자 중 하나를 랜덤하게 샘플링
np.random.choice(10)

3

In [2]:
np.random.choice(10)

4

In [3]:
# words에서 하나만 무작위로 샘플링
words = ['you', 'say', 'goodbye', 'i', 'hello', '.']
np.random.choice(words)

'goodbye'

In [4]:
# 5개만 무작위로 샘플링(중복 없음)
np.random.choice(words, size=5, replace=False)

array(['i', '.', 'goodbye', 'hello', 'say'], dtype='<U7')

In [5]:
# 확률분포에 따라 샘플링
p = [0.5, 0.1, 0.05, 0.2, 0.05, 0.1]
np.random.choice(words, p=p)

'goodbye'

In [9]:
# 0.75를 제곱함으로써, 원래 확률이 낮은 단어의 확률을 살짝 높일 수 있다.

p = [0.7, 0.29, 0.01]
new_p = np.power(p, 0.75)
new_p /= np.sum(new_p)
new_p

array([0.64196878, 0.33150408, 0.02652714])

In [12]:
# Negative Sampling 클래스 구현

import sys
sys.path.append('..')
from common.np import *  # import numpy as np
from common.layers import Embedding, SigmoidWithLoss
import collections

class UnigramSampler:
    def __init__(self, corpus, power, sample_size):
        self.sample_size = sample_size
        self.vocab_size = None
        self.word_p = None

        counts = collections.Counter()
        for word_id in corpus:
            counts[word_id] += 1

        vocab_size = len(counts)
        self.vocab_size = vocab_size

        self.word_p = np.zeros(vocab_size)
        for i in range(vocab_size):
            self.word_p[i] = counts[i]

        self.word_p = np.power(self.word_p, power)
        self.word_p /= np.sum(self.word_p)

    def get_negative_sample(self, target):
        batch_size = target.shape[0]

        if not GPU:
            negative_sample = np.zeros((batch_size, self.sample_size), dtype=np.int32)

            for i in range(batch_size):
                p = self.word_p.copy()
                target_idx = target[i]
                p[target_idx] = 0
                p /= p.sum()
                negative_sample[i, :] = np.random.choice(self.vocab_size, size=self.sample_size, replace=False, p=p)
        else:
            # GPU(cupy）로 계산할 때는 속도를 우선한다.
            # 부정적 예에 타깃이 포함될 수 있다.
            negative_sample = np.random.choice(self.vocab_size, size=(batch_size, self.sample_size),
                                               replace=True, p=self.word_p)

        return negative_sample

- UnigramSampler 클래스는 초기화 시 3개의 인수를 받음.
  - 단어ID목록인 corpus, 확률분포에'제곱'할 값인 power(기본값 0.75), '부정적인 예 샘플링'을 수행하는 횟수인 sample_size.   
- get_negative_sample(target)메서드는 target인수로 지정한 단어를 긍정적인 예로 해석하고, 그 외의 단어ID를 샘플링함.
  

In [13]:
corpus = np.array([0, 1, 2, 3, 4, 1, 2, 3])
power = 0.75
sample_size = 2

sampler = UnigramSampler(corpus, power, sample_size)
target = np.array([1, 3, 0])
negative_sample = sampler.get_negative_sample(target)
print(negative_sample)

[[2 3]
 [2 4]
 [2 4]]


## 4.2.7 네거티브 샘플링 구현

In [14]:
class NegativeSamplingLoss:
    def __init__(self, W, corpus, power=0.75, sample_size=5):
        self.sample_size = sample_size
        self.sampler = UnigramSampler(corpus, power, sample_size)
        self.loss_layers = [SigmoidWithLoss() for _ in range(sample_size + 1)]
        self.embed_dot_layers = [EmbeddingDot(W) for _ in range(sample_size + 1)]

        self.params, self.grads = [], []
        for layer in self.embed_dot_layers:
            self.params += layer.params
            self.grads += layer.grads

    def forward(self, h, target):
        batch_size = target.shape[0]
        negative_sample = self.sampler.get_negative_sample(target)

        # 긍정적 예 순전파
        score = self.embed_dot_layers[0].forward(h, target)
        correct_label = np.ones(batch_size, dtype=np.int32)
        loss = self.loss_layers[0].forward(score, correct_label)

        # 부정적 예 순전파
        negative_label = np.zeros(batch_size, dtype=np.int32)
        for i in range(self.sample_size):
            negative_target = negative_sample[:, i]
            score = self.embed_dot_layers[1 + i].forward(h, negative_target)
            loss += self.loss_layers[1 + i].forward(score, negative_label)

        return loss

    def backward(self, dout=1):
        dh = 0
        for l0, l1 in zip(self.loss_layers, self.embed_dot_layers):
            dscore = l0.backward(dout)
            dh += l1.backward(dscore)

        return dh